In [ ]:
import networkx as nx
import itertools as it
from pyvis.network import Network

We'll make some De Bruijn graphs and look at some observations.

In [ ]:
n = 3
k = 2

In [ ]:
alphabet = [str(_) for _ in range(k)]

In [ ]:
G = nx.DiGraph()

In [ ]:
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
print(nodes)

In [ ]:
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
print(edges)

In [ ]:
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
# Optional. Run if you want an interactive graph drawing.
net = Network(notebook=True, height='600px', width='600px', directed=True)
net.from_nx(G)
net.toggle_physics(False)
net.show_buttons(filter_=['edges'])
net.show('example.html')

<img src="graph.png">

We find all the $k-cycle$ in above graph such that $k < n$.

In [ ]:
cyc = list(nx.algorithms.simple_cycles(G))
for c in cyc:
    if len(c) < n:
        print(c)

Out of all the possible words of length $n$. Only those that are parts of above cycles can be repeated.

We can see similar thing for $n=4$

In [ ]:
n = 4
G = nx.DiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
print(nodes)

In [ ]:
print(edges)

In [ ]:
cyc = list(nx.algorithms.simple_cycles(G))
for c in cyc:
    if len(c) < n:
        print(c)

We see that the words that part of $k-cycle$ such that $k<n$ are the only ones that can be repeated. Also, these are border words.

On of the construction of De Bruijn sequence involve finding a hamiltonian path in a $n$ dimensional De Bruijn graph. We can modify that construction to allow visiting the above mentioned vertices multiple time.

Alternately, we can make $n-1$ dimensional De Bruijn graph and find all the $k-cycle$ with $k \le n$ and instead of finding an Eulerian cycle that passes through all edges once, we find a circuit that passes through all edges exactly once except the edges that are part of mentioned $k-cycle$. Following is the example.

In [ ]:
n = 3
G = nx.DiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
print(nodes)

In [ ]:
print(edges)

In [ ]:
cyc = list(nx.algorithms.simple_cycles(G))
for c in cyc:
    if len(c) <= n:
        print(c)

The notion of repeating edges can be thought of as having multiple edges. For this example what seem to be working is having $4$  edges for $000$ and $111$ cycle and $2$ for the rest of the cycles. So, we can add those edges.

In [ ]:
G = nx.MultiDiGraph()
nodes = [''.join(tup) for tup in list(it.product(alphabet, repeat=n))]
edges = []
for node in nodes:
    for letter in alphabet:
        edges.append((node, node[1:]+letter))
G.add_nodes_from(nodes)
G.add_edges_from(edges)

more_edges = [('000', '000') for _ in range(3)]
G.add_edges_from(more_edges)
more_edges = [('111', '111') for _ in range(3)]
G.add_edges_from(more_edges)
v = ['001', '010', '100']
G.add_edges_from([(v[i], v[(i+1)%len(v)]) for i in range(len(v))])
v = ['011', '110', '101']
G.add_edges_from([(v[i], v[(i+1)%len(v)]) for i in range(len(v))])

In [ ]:
print(G.edges)

In [ ]:
# Exporting and using Cytoscape to view graph as repeated edges are overlapping in pyvis.
nx.write_gml(G, 'g.gml')

<img src="multi.png">

Edges are directed as given in previous graph, for some reason it's not rendering here.
Now we traverse this graph to find the longest trail (repeating vertices but not edges). If we look the corresponding word in paper.
$01010100100110110111111100000001$.
We see that it starts at $010$. And progresses as $010-10100100110110111111100000001$ Here each letter after $010$ indicates the last letter of the vertex we are approaching as per De Bruijn graph.

However, one thing I noticed is we can enter/exit any cycle exactly once. Meaning if we enter a cycle say at $010$ and we can traverce full cycle of $010-100-001$ at most twice, but if at any point, we get out of the cycle, then we cannot traverse a full cycle any more (partial is allowed) upon re-entry.

Example:
Let's start with $010$ we go to finish $010-100-001$ cycle twice, we get $010010010$. Here we have two instances of $0100$, $1001$ and $0010$, none disjoint.

Now, let's try another way, where we tranverse cycle once, to get $010010$, now instead of traversing cycle again, we exit and re-entre that cycle. This can be done by visiting $101$ once, so we get $01001010$. Now since we exit cycle once and came back, we cannot complete a full cycle. We can visit $100$ and $001$ to get $0100101001$, but cannot come back to $010$ as that will complete the cycle and the obtained word will be $01001010010$ with two disjoint instance of $0010$.

So, what I think can be done is we can construct a multiple edges graphs as mentioned and find the longest trail with restriction on cycles as mentioned above. I think that will give us the longest word with our properties. I tried looking for some algorithm that is polynomial time for such operations, so far I think there aren't any. But it's still looks better than brute-force.

I'm not sure at the moment if this will lead us to some breakthrough. I'm currently thinking about some ways to avoid exit/entry into cycle, in terms of graph, or maybe in some other way to translate this problem to something else where is might be easier to work with. Also I'll work over some alteration of Lyndon word construction of De Bruijn sequence, and see if there is something we can do.